In [5]:
# LangChain 라이브러리에서 필요한 클래스들을 임포트합니다.
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# 낮은 온도(temperature)로 설정된 언어 모델(LLM)을 초기화하여 더 결정적인 응답을 생성합니다.
llm = ChatOpenAI(
    temperature=0.1,
)

# 임베딩이나 다른 캐시된 데이터를 저장할 로컬 캐시 디렉터리를 설정합니다.
cache_dir = LocalFileStore("./.cache/")

# 문서를 작은 청크로 나누기 위한 텍스트 분할기를 생성합니다.
# 여기서는 토큰을 기준으로 분할하며, 각 청크는 최대 600 토큰,
# 청크 간 100 토큰이 겹치도록 설정합니다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 파일에서 비정형 문서(예: 일반 텍스트)를 로드합니다.
# 이 경우 "chapter_one.txt" 파일을 로드합니다.
loader = UnstructuredFileLoader("./files/chapter_one.txt")

# 로드한 문서를 이전에 정의한 분할기를 사용해 작은 청크로 나눕니다.
docs = loader.load_and_split(text_splitter=splitter)

# 문서 청크를 의미적 유사성 분석을 위해 벡터로 변환하기 위한 OpenAI 임베딩을 초기화합니다.
embeddings = OpenAIEmbeddings()

# 임베딩을 캐시로 감싸 동일한 텍스트에 대해 임베딩을 다시 계산하지 않도록 합니다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 문서 청크와 캐시된 임베딩을 사용해 FAISS 벡터 저장소를 생성합니다.
# 이를 통해 문서 청크 간의 유사성을 효율적으로 검색할 수 있습니다.
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 벡터 저장소에서 질의에 따라 관련 문서를 검색할 수 있는 리트리버를 생성합니다.
retriever = vectorstore.as_retriever()

# 각 문서 청크를 처리하기 위한 첫 번째 단계로 프롬프트 템플릿을 정의합니다.
# 이 프롬프트는 문서의 일부가 질문에 관련이 있는지 확인하는 데 사용됩니다.
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 위의 프롬프트와 LLM을 사용하여 각 문서 청크를 처리하는 체인을 생성합니다.
map_doc_chain = map_doc_prompt | llm

# `map_doc_chain`을 각 문서 청크에 적용하는 함수를 정의합니다.
# 이 함수는 문서 청크 목록과 질문을 입력으로 받아, 관련된 모든 부분을 하나의 문자열로 반환합니다.
def map_docs(inputs):
    documents = inputs["documents"]  # 입력 딕셔너리에서 문서를 추출합니다.
    question = inputs["question"]  # 입력 딕셔너리에서 질문을 추출합니다.
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents  # 각 문서 청크에 체인을 적용합니다.
    )

# 리트리버와 map_docs 함수를 사용해 문서에서 관련 부분을 맵핑하는 체인을 정의합니다.
map_chain = {
    "documents": retriever,  # 리트리버가 관련 문서 청크를 가져옵니다.
    "question": RunnablePassthrough(),  # 질문을 수정 없이 그대로 전달합니다.
} | RunnableLambda(map_docs)  # 검색된 문서에 map_docs 함수를 적용합니다.

# 추출된 문서 부분을 바탕으로 최종 답변을 생성하기 위한 최종 프롬프트 템플릿을 정의합니다.
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)
# map_chain과 final_prompt를 결합하여 전체 처리 체인을 만듭니다.
# 이 체인은 질문을 받아 관련 문서 부분을 검색하고, 최종 답변을 생성합니다.
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

# 특정 질문으로 체인을 호출합니다.
# 체인은 문서를 처리하여 관련 정보를 찾고, 최종 답변을 생성합니다.
chain.invoke("How many ministries are mentioned?")


AIMessage(content='Victory Mansions is a building where Winston Smith lives. It is a run-down apartment complex with glass doors, gritty dust, and a hallway that smells of boiled cabbage and old rag mats. The building has a faulty lift, and Winston\'s flat is seven flights up. Inside the flat, there is a telescreen that cannot be completely shut off, and a poster with a large face that reads "BIG BROTHER IS WATCHING YOU."')